Load API Keys

In [94]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [95]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html"
])
data = loader.load()

Split the Loaded Data into Chunks

In [96]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)

Initialize Pinecone

In [97]:
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "hlb-test"

Create Embeddings and Vectors to Insert to Pinecone Database

In [98]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

Create Retriever for Vector Search

In [99]:
retriever = vector.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5})

Creating Prompt Template

In [108]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  Reply to the following input: {input}
  By only referring the follwing document: {document}
  You have access to the current chat's history: {chat_history}
  If you are not sure about deposit type, you can ask for clarification.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the input, answer with the assumption that the question is about HLB.
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'document', 'chat_history'])

Storing History and Creating Chain

In [109]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", input_key="input")
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

Running the Chain

In [113]:
query = "How much will RM550 become for 3 months of fixed deposit?"
document = retriever.get_relevant_documents(query)
chain.run(input=query, document=document)

'The fixed deposit interest rate for 3 months at HLB is 2.50% per annum. Therefore, RM550 will become RM557.92 after 3 months of fixed deposit.'